In [ ]:
%matplotlib inline

In [ ]:
# !pip install pandas_summary
# !pip install waterfallcharts
# !pip install treeinterpreter
# !pip install time

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import waterfall_chart

from fastai.imports import *
from fastai.tabular import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_recall_curve, confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from matplotlib.ticker import FuncFormatter


# Data Processing

In [ ]:
#load data
cancer = load_breast_cancer()

In [ ]:
cancer.target_names

In [ ]:
data = np.c_[cancer.data, cancer.target]
columns = np.append(cancer.feature_names, ["target"])
cancer_df=pd.DataFrame(data, columns=columns)

In [ ]:
cancer_df.describe()

Show class distribution of target variable

In [ ]:
unique, counts = np.unique(cancer_df["target"], return_counts=True)
plt.pie(counts, labels=unique, autopct='%.0f%%');

In [ ]:
cancer_df.shape

There are 569 rows and 31 columns. 

#### Check for nulls

In [ ]:
# check for missing values
cancer_df.columns[cancer_df.isnull().any()]

No missing values

# Random forest first try

#### Train-Test Split

In [ ]:
#split data into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(cancer_df.iloc[:,cancer_df.columns != 'target'],
                                                    cancer_df['target'],
                                                    test_size=0.25, 
                                                    random_state=100)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

We create the fist RF classifier only to find later optimal parameters <BR>(this would be just Bagging)

In [ ]:
baseRF = RandomForestClassifier(n_estimators = 200,  #The number of trees in the forest
                               random_state = 0,
                               n_jobs = -1, # The number of jobs to run in parallel (-1 means all processors, 1 no parallelism)
                               oob_score = True) # Whether to use out-of-bag samples to estimate the generalization score (increases time)
# oob are the samples not chosen on the boostrapping process

# other parameters:
# criterion{"gini", "entropy", "log_loss"}, default="gini"
# max_depth: int, default=None
# min_samples_leaf: int or float, default=1 - The minimum number of samples required to be at a leaf node
# min_samples_split: int or float, default=2 - The minimum number of samples required to split an internal node:
# max_features{"sqrt", "log2", None}, int or float, default="sqrt" - The number of features to consider when looking for the best split

Now we are searching for the set of parameters that give the best performance, then we will need to use them in a RF Classifier

In [ ]:
param_grid = {
    'min_samples_leaf' :[1,3,5], 
    'max_features' : [10,15,20,25,30],
    'criterion' : ['gini','entropy'] #,'log_loss']
}

The following execution will take some time (1 to 2 minutes). Ignore the warnings

In [ ]:
import time 
start_time = time.time()

from sklearn.model_selection import GridSearchCV
cvRF = GridSearchCV(estimator=baseRF, param_grid=param_grid, cv=3, scoring='roc_auc') 
#cv is the number of cross validation iterations to be performed
cvRF.fit(x_train,y_train)

print ("Completed in --- %s seconds ---" % (time.time() - start_time))

In [ ]:
# shows the best combination of criteria found
cvRF.best_params_

In [ ]:
# shows the best score (of the scoring function we selected, here ROC AUC) from TRAINING DATASET
cvRF.best_score_

Our best parameters are {'criterion': 'entropy', 'max_features': 15, 'min_samples_leaf': 5}.<BR>
Our average AUC score of all folds for training dataset is 0.989.

#### Now we train the model with the best parameters and using Random Forest instead of simple Bagging.
##### The difference is the parameter "max_features"

In [ ]:
model = RandomForestClassifier(n_estimators = 200, 
                               random_state = 0,
                               max_features = 15, #this parameter makes the difference between simple Bagging and Random Forests
                               n_jobs = -1,
                               oob_score = True,
                               criterion = 'entropy',
                               min_samples_leaf = 5)
model.fit(x_train,y_train)

# predictions
y_pred_test = model.predict(x_test)

In [ ]:
unique, counts = np.unique(y_pred_test, return_counts=True)
plt.pie(counts, labels=unique, autopct='%.0f%%');

In [ ]:
print("OOB score: {:.3f}".format(model.oob_score_))
# The OOB score is an estimate of the generalization error of the model, 
# calculated with the data not used to train it.

In [ ]:
# attributes of the model object (for your reference)
# print(dir(model))

Now we are applying our model to the TEST dataset

In [ ]:
pred_prob = model.predict_proba(x_test) #probability of being in both groups
prob_malign = [p[1] for p in pred_prob] #probability of being in the malign group
auc = roc_auc_score(y_test, prob_malign)

# https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
print("AUC score: {:.3f}".format(auc))

Our auc score is very good for this model. <BR>
Let's compare accuracy and show ROC curve

In [ ]:
# AUC Calculations - false positive rates, true positive rates and thresholds
fpr, tpr, thresholds = metrics.roc_curve(y_test, prob_malign, pos_label=1)

train_acc = round(model.score(x_train,y_train) * 100,2) #Train Accuracy score
test_acc = round(model.score(x_test,y_test) * 100,2) #Test Accuracy score
print("Train Accuracy score: ", train_acc, "%")
print("Test Accuracy score: ", test_acc, "%")

In [ ]:
# ROC curve

fig, ax = plt.subplots(1, 1)
ax.plot(fpr, tpr, label="ROC")
ax.plot([0, 1], [0, 1],linestyle='--', label="Random model") 
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
ax.set_box_aspect(1)
ax.legend()
plt.show()

And now the Precision and Recall chart

In [ ]:
# We can compare Recall and Precision

prob_pred = model.predict_proba(x_test)[:, 1]
thresholds = np.arange(0.0, 1.0, step=0.01)
recall_scores = [metrics.recall_score(y_test, prob_pred > t) for t in thresholds]
precis_scores = [metrics.precision_score(y_test, prob_pred > t) for t in thresholds]
fig, ax = plt.subplots(1, 1)
ax.plot(thresholds, recall_scores, label="Recall @ t")
ax.plot(thresholds, precis_scores, label="Precision @ t")
ax.axvline(0.5, c="gray", linestyle="--", label="Default Threshold")
ax.set_xlabel("Threshold")
ax.set_ylabel("Metric @ Threshold")
ax.set_box_aspect(1)
ax.legend()
plt.show()

### Tree interpreter: calculate the contribution of each feature to the tree prediction.
##### We will use it first with scikit and then with an additional library

In [ ]:
dir(model)
model.feature_names_in_

In [ ]:
importances = model.feature_importances_

indexes = np.argsort(importances)[::-1]
sorted_imp = importances[indexes]

plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(importances)), importances[indexes], align='center')
plt.xticks(range(len(importances)), np.array(model.feature_names_in_)[indexes], rotation=90)
plt.xlabel("Features")
plt.ylabel("Importance")
plt.title("Feature importances")
for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{sorted_imp[i]:.2f}', ha='center', va='bottom')
plt.show()

##### Now with an additional library that shows individual importance directions and decomposes the model prediction into three parts: 
1. Bias: The overall average prediction of the model without taking individual features into account.
2. Contributions: The specific influence of each feature on the prediction value for the instance in question.
3. Final Prediction: This is the final result for the particular observation, obtained by summing the bias and all feature contributions.

In [ ]:
from treeinterpreter import treeinterpreter as ti

In [ ]:
predictions, biases, contributions = ti.predict(model, x_test.values)

# Convert contributions into a dataframe
contributions_df = pd.DataFrame(contributions[:, :, 0], columns=x_test.columns)

# Suming contributions from every row
total_contributions = contributions_df.sum(axis=0)

# Show chart (now displaying features with contribution less than 0.03)
waterfall_chart.plot(x_test.columns, total_contributions, rotation_value=90, threshold=0.03, formatting='{:,.2f}')
plt.title('Waterfall Chart of Total Contributions')
plt.xlabel('Features')
plt.ylabel('Contributions')
plt.show()

### What is the optimum number of trees?

Overfitting does not depend on the number of trees, but we don´t want to run with more trees than necessary

In [ ]:
train_scores = []
cv_scores    = []

estimator_range = range(5, 105, 5) #from 5 trees to 100, step 5

for n_estimators in estimator_range:
    model = RandomForestClassifier(
                n_estimators = n_estimators,
                max_features = 15, # the previous optimum
                oob_score    = False,
                n_jobs       = -1, 
                random_state = 123
             )
    
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    acc=accuracy_score(y_test, predictions)
    print("% Accuracy of test dataset for {} trees is {:.3f}".format(n_estimators, acc))
    train_scores.append(acc) # storing accuracy from each iteration

    # and we also store the accuracy mean obtained from running a 5-fold validation
    cvscores = cross_val_score(
                estimator = model,
                X         = x_train,
                y         = y_train,
                scoring   = 'accuracy',
                cv        = 5
             )
    cv_scores.append(cvscores.mean()) 

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(estimator_range, train_scores, marker='o', linestyle='-', color='b')
plt.title('Accuracy of Test Dataset vs. Number of Trees')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Accuracy')
plt.xticks(estimator_range)  # Para mostrar todos los valores en el eje x
plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, cv_scores, label="cv scores")
ax.plot(estimator_range[np.argmax(cv_scores)], max(cv_scores),
        marker='o', color = "red", label="max score")
ax.set_ylim(0.93, 0.97)
ax.set_ylabel("% Accuracy")
ax.set_xlabel("n_estimators")
ax.set_title("Evolution of cv-error vs. number of trees")
plt.legend();
print(f"Optimum number of trees: {estimator_range[np.argmax(cv_scores)]}")